## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,56.48,37,11,5.39,few clouds
1,1,Carnarvon,AU,-24.8667,113.6333,93.27,33,0,17.27,clear sky
2,2,Esperance,AU,-33.8667,121.9000,75.04,51,0,6.87,clear sky
3,3,Punta Arenas,CL,-53.1500,-70.9167,42.91,70,40,24.16,scattered clouds
4,4,Iracoubo,GF,5.4802,-53.2011,75.31,91,99,9.53,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Esperance,AU,-33.8667,121.9000,75.04,51,0,6.87,clear sky
4,4,Iracoubo,GF,5.4802,-53.2011,75.31,91,99,9.53,light rain
9,9,New Norfolk,AU,-42.7826,147.0587,81.66,37,92,3.60,overcast clouds
10,10,Hobart,AU,-42.8794,147.3294,82.02,51,75,6.91,broken clouds
13,13,Busselton,AU,-33.6500,115.3333,83.08,45,0,8.63,clear sky
17,17,Cabo San Lucas,MX,22.8909,-109.9124,83.50,62,0,11.05,clear sky
19,19,Galle,LK,6.0367,80.2170,77.49,80,97,2.59,overcast clouds
24,24,Nueva Granada,CO,9.8017,-74.3930,83.32,48,100,7.23,overcast clouds
26,26,Atuona,PF,-9.8000,-139.0333,77.79,77,6,13.69,clear sky
27,27,Mar Del Plata,AR,-38.0023,-57.5575,80.62,54,0,11.50,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                172
City                   172
Country                172
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Esperance,AU,75.04,clear sky,-33.8667,121.9000,
4,Iracoubo,GF,75.31,light rain,5.4802,-53.2011,
9,New Norfolk,AU,81.66,overcast clouds,-42.7826,147.0587,
10,Hobart,AU,82.02,broken clouds,-42.8794,147.3294,
13,Busselton,AU,83.08,clear sky,-33.6500,115.3333,
17,Cabo San Lucas,MX,83.50,clear sky,22.8909,-109.9124,
19,Galle,LK,77.49,overcast clouds,6.0367,80.2170,
24,Nueva Granada,CO,83.32,overcast clouds,9.8017,-74.3930,
26,Atuona,PF,77.79,clear sky,-9.8000,-139.0333,
27,Mar Del Plata,AR,80.62,clear sky,-38.0023,-57.5575,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Esperance,AU,75.04,clear sky,-33.8667,121.9000,Hospitality Esperance
4,Iracoubo,GF,75.31,light rain,5.4802,-53.2011,Le Gîte d’Iracoubo
9,New Norfolk,AU,81.66,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
10,Hobart,AU,82.02,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
13,Busselton,AU,83.08,clear sky,-33.6500,115.3333,Observatory Guest House
17,Cabo San Lucas,MX,83.50,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
19,Galle,LK,77.49,overcast clouds,6.0367,80.2170,Amangalla
24,Nueva Granada,CO,83.32,overcast clouds,9.8017,-74.3930,HOTEL BELLA VISTA NUEVA GRANADA
26,Atuona,PF,77.79,clear sky,-9.8000,-139.0333,Villa Enata
27,Mar Del Plata,AR,80.62,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata


In [18]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.dropna(axis=0, how='any')

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Esperance,AU,75.04,clear sky,-33.8667,121.9000,Hospitality Esperance
4,Iracoubo,GF,75.31,light rain,5.4802,-53.2011,Le Gîte d’Iracoubo
9,New Norfolk,AU,81.66,overcast clouds,-42.7826,147.0587,The Shingles Riverside Cottages
10,Hobart,AU,82.02,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
13,Busselton,AU,83.08,clear sky,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
641,Koumac,NC,84.70,scattered clouds,-20.5667,164.2833,Le Monitel
645,Minna,NG,79.66,clear sky,9.6139,6.5569,Royal Park hotel & suites
646,Tiarei,PF,89.62,clear sky,-17.5333,-149.3333,Le Rocher de Tahiti
649,Serra Talhada,BR,80.35,overcast clouds,-7.9919,-38.2983,Pousada São Luiz


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")


In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))